AIFactory의 Langchain 무료강의: https://www.youtube.com/watch?v=2yv4PxE1Ks0  
2강: https://colab.research.google.com/drive/1FRtKBT28ZZops5jtOvsRWpL_UsK2D1fz#scrollTo=C5uYZQknXLDM  
3강: https://colab.research.google.com/drive/1yBOOmPcmPLO-REQMy4Of4T0Wa9GfBHpy#scrollTo=Yl_SXA37i4p5  
6강: https://colab.research.google.com/drive/1w5_n2747R4os9CH5R-2uiXT3ftvuBwAj

# 1강. 그라디오 챗봇 시작하기

In [1]:
import gradio as gr

/Users/limkahyun/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def greet(name):
    return "안녕! " + name

In [6]:
demo = gr.Interface(fn=greet,
                    inputs='text',
                    outputs='text'
                   )

In [7]:
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


# 2강. openAI API 사용해보기

In [8]:
import os
import openai

In [18]:
# 환경변수에서 OPENAI_API_KEY를 가져와 openai.api_key에 입력
os.environ['OPENAI_API_KEY'] = '-'
openai.api_key = os.getenv('OPENAI_API_KEY')

In [20]:
response = openai.Completion.create(model='text-davinci-003',   # 사용할 AI 모델 지정
                                    prompt='안녕? 넌 누구니?',     # 모델에게 전달할 프롬프트 지정
                                    max_tokens=256,            # 생성될 텍스트의 최대 토큰수 지정
                                    temperature=0              # 0:가장 확률이 높은 텍스트 생성 1:더 다양한 텍스트 생성
                                   )

In [22]:
print(response)

{
  "id": "cmpl-7bPEIRlQtYLq4TvbJBwZX5Re1YOhH",
  "object": "text_completion",
  "created": 1689149474,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n\n\uc548\ub155\ud558\uc138\uc694. \ub098\ub294 \ud64d\uae38\ub3d9\uc774\uc5d0\uc694.",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 19,
    "completion_tokens": 42,
    "total_tokens": 61
  }
}


# 3강. openAPI와 그라디오 챗봇 연동하기

In [4]:
import os
import openai
import gradio as gr

In [7]:
# 환경변수에서 OPENAI_API_KEY를 가져와 openai.api_key에 입력
os.environ['OPENAI_API_KEY'] = '-'
openai.api_key = os.getenv('OPENAI_API_KEY')

In [8]:
def respond(message, chat_history):  # 채팅봇의 응답을 처리하는 함수를 정의합니다.
    
    response = openai.Completion.create(model="text-davinci-003",
                                        prompt=message,
                                        max_tokens=1000,
                                        temperature=0
                                       )
    
    # json에서 출력 텍스트를 저장
    bot_message = response.choices[0].text 
    
    # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가합니다.
    chat_history.append((message, bot_message))

    return "", chat_history  # 수정된 채팅 기록을 반환합니다.

In [9]:
# 그라디오 UI를 이용하기
# gr.Blocks()를 사용하여 인터페이스를 생성
with gr.Blocks() as demo:
    # '채팅창'이라는 레이블을 가진 채팅봇 컴포넌트를 생성합니다.
    chatbot = gr.Chatbot(label="채팅창")
    
    # '입력'이라는 레이블을 가진 텍스트박스를 생성합니다.
    msg = gr.Textbox(label="입력")  
    
    # '초기화'라는 레이블을 가진 버튼을 생성합니다.
    clear = gr.Button("초기화")
    
    # 텍스트박스에 메시지를 입력하고 제출하면 respond 함수가 호출, [msg, chatbot]을 respond함수에 입력, respond 함수의 return도 [msg, chatbot]
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    
    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화합니다.
    clear.click(lambda: None, None, chatbot, queue=False)  

# 챗봇을 주고받으려면 그라디오가 계속 실행되기 때문에 종료하려면 정지버튼 누르기
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


# 4강. 랭체인 LLM과 그라디오 챗봇 연동하기
openai를 langchain LLM으로 교체

In [10]:
import os
from langchain.llms import OpenAI
import gradio as gr

In [11]:
# 환경변수에서 OPENAI_API_KEY를 가져와 openai.api_key에 입력
os.environ['OPENAI_API_KEY'] = '-'
openai.api_key = os.getenv('OPENAI_API_KEY')

In [18]:
llm = OpenAI(model_name='text-davinci-003', temperature=0.1)

In [13]:
llm('안녕? 넌 누구니?')

'\n\n안녕하세요. 나는 친구입니다.'

In [19]:
# 그라디오 연결
def respond(message, chat_history):
    bot_message = llm(message)
    chat_history.append([message, bot_message])
    return '', chat_history

In [20]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label='chatting')
    msg = gr.Textbox(label='input')
    clear = gr.Button('reset')
    
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

In [21]:
# 챗봇을 주고받으려면 그라디오가 계속 실행되기 때문에 종료하려면 정지버튼 누르기
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


# 5강. 랭체인 챗모델과 그라디오 챗봇 연동하기
LLM모델에서 chatgpt 모델로 교체

In [3]:
import os
import openai
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import gradio as gr

In [4]:
# 환경변수에서 OPENAI_API_KEY를 가져와 openai.api_key에 입력
os.environ['OPENAI_API_KEY'] = '-'
openai.api_key = os.getenv('OPENAI_API_KEY')

In [5]:
chat_llm = ChatOpenAI(model='gpt-3.5-turbo')

In [6]:
result = chat_llm([HumanMessage(content='안녕? 넌 누구니?')])

In [7]:
result.content

'안녕! 나는 OpenAI의 인공지능 모델이야. 너에게 어떻게 도움을 줄까?'

In [8]:
# 그라디오 연결
def respond(message, chat_history):
    result = chat_llm([HumanMessage(content=message)])
    bot_message = result.content
    chat_history.append([message, bot_message])
    return '', chat_history

In [10]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label='채팅창')
    msg = gr.Textbox(label='입력')
    clear = gr.Button('초기화')
    
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

In [11]:
# 다음셀 사용하려면 정지버튼 누르기
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


# 6강. pdf기반 질의응답 챗봇
huggingface embedding을 사용하면 비용 발생하지 않음  
huggingface embedding을 사용하기 위해서는 아래와 같이 설치 필요
- pip install huggingface_hub
- pip install sentence_transformers

In [1]:
import os

# pdf 벡터화
from langchain.document_loaders import PyPDFLoader
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

# 랭체인 챗봇
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

# 프롬프트 한국어로 설정
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 그라디오
import gradio as gr

/Users/limkahyun/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ChatOpenAI 사용하기 위함
os.environ["OPENAI_API_KEY"] = "-"

In [3]:
# pdf샘플로 회사 브로셔 사용
loader = PyPDFLoader("2023_kubwa.pdf")
documents = loader.load()

# 파일 안에 있는 text를 일정 단위로 달라서 chunk 만들기
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [4]:
texts[0]

Document(page_content='Manufacturing, Industrial, Healthcare 분야�데이터에�특화된 \n컨설팅, 솔루션, 분석�플랫폼을�제공합니다. \nOUR PRODUCT \n& SERVICE\n01. kubwa Company\n02. kubwa Manufacturing\n03. kubwa AIoT\n04. kubwa MLOps\n05. kubwa Use Case', metadata={'source': '2023_kubwa.pdf', 'page': 1})

In [5]:
# 벡터화
# embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings()

# 벡터 저장
vector_store = Chroma.from_documents(texts, embeddings)

# 벡터 유사도 관련성 중 k를 지정해 상위값 검색할 수 있게 설정
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [6]:
# openAI 불러오기
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

In [7]:
# 체인 만들기 (문서기반 질의응답 할 수 있는 체인)
chain = RetrievalQAWithSourcesChain.from_chain_type(llm=llm,
                                                    chain_type='stuff',
                                                    retriever = retriever,
                                                    return_source_documents=True
                                                   )

In [8]:
q = 'what is kubwa?'
result = chain(q)
print(result)
# result['answer'] : 랭체인의 답변
# result['source'] : 답변의 근거

{'question': 'what is kubwa?', 'answer': 'Kubwa is a company that provides consulting, solutions, and analysis platforms specialized in data for the manufacturing, industrial, and healthcare sectors. It aims to create a large AI ecosystem (ecosystem) similar to the forests of the African savannah to contribute. Kubwa is derived from the Swahili word for "BIG." \n', 'sources': '2023_kubwa.pdf', 'source_documents': [Document(page_content='address  경기도�성남시�판교역로 230, 삼환하이펙스 B동 207호\nwebsite  www.kubwa.co.kr   e-mail  kubwa@kubwa.co.kr쿱와(kubwa) 는 아프리카 스와힐리어로 BIG을 의미합니다.\n쿱와는 아프리카 초원지대의 숲처럼 커다란 AI 생태계(에코시스템)를 만들어 기여하고자 합니다.', metadata={'source': '2023_kubwa.pdf', 'page': 7}), Document(page_content='Manufacturing, Industrial, Healthcare 분야�데이터에�특화된 \n컨설팅, 솔루션, 분석�플랫폼을�제공합니다. \nOUR PRODUCT \n& SERVICE\n01. kubwa Company\n02. kubwa Manufacturing\n03. kubwa AIoT\n04. kubwa MLOps\n05. kubwa Use Case', metadata={'source': '2023_kubwa.pdf', 'page': 1})]}


In [7]:
# 프롬프트 한국어로 설정
system_template="""Use the following pieces of context to answer the users question shortly.
Given the following summaries of a long document and a question, create a final answer with references ("SOURCES"),
use "SOURCES" in capital letters regardless of the number of sources.
If you don't know the answer, just say that "I don't know", don't try to make up an answer.
----------------
{summaries}
You MUST answer in Korean and in Markdown format:"""

messages = [SystemMessagePromptTemplate.from_template(system_template),
            HumanMessagePromptTemplate.from_template("{question}")
           ]

prompt = ChatPromptTemplate.from_messages(messages)

In [8]:
# 체인 만들기 (한국어 설정)
chain = RetrievalQAWithSourcesChain.from_chain_type(llm=llm,
                                                    chain_type='stuff',
                                                    retriever=retriever,
                                                    return_source_documents=True,
                                                    chain_type_kwargs={'prompt':prompt}
                                                   )

In [9]:
q = '회사 이름이 뭐야?'
result = chain(q)
print(result['answer'])

회사 이름은 쿱와(kubwa)입니다. [SOURCE]


In [10]:
# 내용확인
for doc in result['source_documents']:
    print('파일: ', doc.metadata['source'])
    print('내용: ', doc.page_content[0:100].replace('\n', ' '))
    print('페이지: ', doc.metadata['page'])

파일:  2023_kubwa.pdf
내용:  address  경기도�성남시�판교역로 230, 삼환하이펙스 B동 207호 website  www.kubwa.co.kr   e-mail  kubwa@kubwa.co.kr쿱와(kub
페이지:  7
파일:  2023_kubwa.pdf
내용:  Manufacturing, Industrial, Healthcare 분야�데이터에�특화된  컨설팅, 솔루션, 분석�플랫폼을�제공합니다.  OUR PRODUCT  & SERVICE 
페이지:  1


In [11]:
# 그라디오 연결
def respond(message, chat_history):

    result = chain(message)
    bot_message = result['answer']

    for i, doc in enumerate(result['source_documents']):
        bot_message += '\n답변출처' + str(i+1) + ') ' + doc.metadata['source'] + '/ page' + str(doc.metadata['page'])

    chat_history.append((message, bot_message))

    return "", chat_history

In [20]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="채팅창").style(height=500)
    msg = gr.Textbox(label="입력")
    clear = gr.Button("초기화")

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

/var/folders/bw/hc1k__h51gsbssz_qpxf16jr0000gn/T/ipykernel_1242/2704861026.py:2: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  chatbot = gr.Chatbot(label="채팅창").style(height=500)


In [ ]:
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
